In [1]:
import time
import pandas as pd
import os

In [2]:
from selenium import webdriver

In [3]:
# 기사정보를 저장하는 함수
def article_save(query1, query2, query3, ds, de):
    # 기사제목, 링크주소, 좋아요, 화나요를 저장할 딕셔너리
    article_dict = { 'title':[], 'link':[], 'like':[], 'angry':[] }
    # cnt : 페이지 변수
    cnt = 1
    # article_search()에서 입력한 조건으로 기사를 검색한다.
    # 한 페이지당 큰 제목 기사 10개씩
    # 네이버는 4000개까지만 제공
    for start in range(1,3992,10) :
        site = f'https://search.naver.com/search.naver?where=news&query={query1}%26{query2}%26{query3}&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={ds}&de={de}&docid=&nso=so%3Ar%2Cp%3Afrom20190101to20200206%2Ca%3Aall&mynews=0&&start={start}&related=0'
        driver.get(site)
        
        # j : 네이버링크 인덱스변수
        j = 0
        for i in range(0, 10, 1) :
            # 기사 내용 중에 (큰 제목만 탐색)
            # 네이버링크가 있으면 
            # 딕셔너리에 기사제목과 링크주소 추가 후
            # 기사를 열어본다.
            content_list = driver.find_elements_by_css_selector('ul.type01 > li > dl')
            content = content_list[i]
            # 링크 인덱스 접근은 기사제목과 독립적으로 한다.
            link_list = driver.find_elements_by_css_selector('ul.type01 > li > dl > dd.txt_inline > a')
            link = link_list[j]
            if ('네이버뉴스' in content.text) == True :
                # 기사 제목과 링크 딕셔너리에 추가
                title = content.find_element_by_css_selector('dt')
                article_dict['title'].append(title.text.strip(''))
                article_dict['link'].append(link.get_attribute('href'))
                link.click()
                # 링크 인덱스가 마지막이 아니면 다음 링크 접근
                if j < (len(link_list)-1) :
                    j += 1
                # 마지막이라면 인덱스 변수 증가 안함
                else :
                    pass
                
                time.sleep(1)
                
                # 네이버뉴스가 열린 탭을 탐색한다.
                news_tab = driver.window_handles[-1]
                driver.switch_to.window(window_name=news_tab)
                
                try:
                    # 좋아요 수와 화나요 수를 탐색한다.
                    like = driver.find_element_by_css_selector('#spiLayer > div._reactionModule.u_likeit > ul > li.u_likeit_list.good > a > span.u_likeit_list_count._count')
                    angry = driver.find_element_by_css_selector('#spiLayer > div._reactionModule.u_likeit > ul > li.u_likeit_list.angry > a > span.u_likeit_list_count._count')
                
                except:
                    # 좋아요와 화나요가 없으면 0으로 추가
                    article_dict['like'].append('0')
                    article_dict['angry'].append('0')
                
                else:
                    # 딕셔너리에 좋아요 수와 화나요 수를 추가한다.(1000이상이면 콤마 제거)
                    article_dict['like'].append(like.text.replace('[\,]', ''))
                    article_dict['angry'].append(angry.text.replace('[\,]', ''))
                # 공감 버튼 유무와 관계없이 실행
                finally:
                    # 기사 탭을 닫고 검색결과 화면으로 돌아간다.
                    driver.close()
                    first_tab = driver.window_handles[0]
                    driver.switch_to.window(window_name=first_tab)
                    
        # 내부for문 종료
    
        # 데이터프레임으로 추가
        df = pd.DataFrame.from_dict(article_dict)
        df.to_csv('naver_article.csv', encoding='utf-8-sig', mode='w', index=False)
        print(f'{cnt}페이지 탐색 완료')
        cnt += 1
        
    # 외부 for문 종료
    return print('\n수집완료')

In [4]:
# 네이버뉴스를 검색하는 함수
def article_search():
    # 검색어, 기간을 입력한다.
    # 날짜 형식 : yyyy.mm.dd
    query1 = input('키워드1 : ')
    query2 = input('키워드2 : ')
    query3 = input('키워드3 : ')
    ds = input('시작일 : ')
    de = input('종료일 : ')
    
    # 기사 저장하는 함수 호출
    article_save(query1, query2, query3, ds, de)

In [5]:
driver = webdriver.Chrome()

article_search()

키워드1 : 강남구
키워드2 : 재개발
키워드3 : 재건축
시작일 : 2016.01.01
종료일 : 2019.12.31
1페이지 탐색 완료
2페이지 탐색 완료
3페이지 탐색 완료
4페이지 탐색 완료
5페이지 탐색 완료
6페이지 탐색 완료
7페이지 탐색 완료
8페이지 탐색 완료
9페이지 탐색 완료
10페이지 탐색 완료
11페이지 탐색 완료
12페이지 탐색 완료
13페이지 탐색 완료
14페이지 탐색 완료
15페이지 탐색 완료
16페이지 탐색 완료
17페이지 탐색 완료
18페이지 탐색 완료
19페이지 탐색 완료
20페이지 탐색 완료
21페이지 탐색 완료
22페이지 탐색 완료
23페이지 탐색 완료
24페이지 탐색 완료
25페이지 탐색 완료
26페이지 탐색 완료
27페이지 탐색 완료
28페이지 탐색 완료
29페이지 탐색 완료
30페이지 탐색 완료
31페이지 탐색 완료
32페이지 탐색 완료
33페이지 탐색 완료
34페이지 탐색 완료
35페이지 탐색 완료
36페이지 탐색 완료
37페이지 탐색 완료
38페이지 탐색 완료
39페이지 탐색 완료
40페이지 탐색 완료
41페이지 탐색 완료
42페이지 탐색 완료
43페이지 탐색 완료
44페이지 탐색 완료
45페이지 탐색 완료
46페이지 탐색 완료
47페이지 탐색 완료
48페이지 탐색 완료
49페이지 탐색 완료
50페이지 탐색 완료
51페이지 탐색 완료
52페이지 탐색 완료
53페이지 탐색 완료
54페이지 탐색 완료
55페이지 탐색 완료
56페이지 탐색 완료
57페이지 탐색 완료
58페이지 탐색 완료
59페이지 탐색 완료
60페이지 탐색 완료
61페이지 탐색 완료
62페이지 탐색 완료
63페이지 탐색 완료
64페이지 탐색 완료
65페이지 탐색 완료
66페이지 탐색 완료
67페이지 탐색 완료
68페이지 탐색 완료
69페이지 탐색 완료
70페이지 탐색 완료
71페이지 탐색 완료
72페이지 탐색 완료
73페이지 탐색 완료
74페이지 탐색 완료
75페이지 탐색 완료
76페이지 탐색 완료
77페이지 탐색 완료
78페이지 탐색 완료
79페이지 

635페이지 탐색 완료
636페이지 탐색 완료
637페이지 탐색 완료
638페이지 탐색 완료
639페이지 탐색 완료
640페이지 탐색 완료
641페이지 탐색 완료
642페이지 탐색 완료
643페이지 탐색 완료
644페이지 탐색 완료
645페이지 탐색 완료
646페이지 탐색 완료
647페이지 탐색 완료
648페이지 탐색 완료
649페이지 탐색 완료
650페이지 탐색 완료
651페이지 탐색 완료
652페이지 탐색 완료
653페이지 탐색 완료
654페이지 탐색 완료
655페이지 탐색 완료
656페이지 탐색 완료
657페이지 탐색 완료
658페이지 탐색 완료
659페이지 탐색 완료
660페이지 탐색 완료
661페이지 탐색 완료
662페이지 탐색 완료
663페이지 탐색 완료
664페이지 탐색 완료
665페이지 탐색 완료
666페이지 탐색 완료
667페이지 탐색 완료
668페이지 탐색 완료
669페이지 탐색 완료
670페이지 탐색 완료
671페이지 탐색 완료
672페이지 탐색 완료
673페이지 탐색 완료
674페이지 탐색 완료
675페이지 탐색 완료
676페이지 탐색 완료
677페이지 탐색 완료
678페이지 탐색 완료
679페이지 탐색 완료
680페이지 탐색 완료
681페이지 탐색 완료
682페이지 탐색 완료
683페이지 탐색 완료
684페이지 탐색 완료
685페이지 탐색 완료
686페이지 탐색 완료
687페이지 탐색 완료
688페이지 탐색 완료
689페이지 탐색 완료
690페이지 탐색 완료
691페이지 탐색 완료
692페이지 탐색 완료
693페이지 탐색 완료
694페이지 탐색 완료
695페이지 탐색 완료
696페이지 탐색 완료
697페이지 탐색 완료
698페이지 탐색 완료
699페이지 탐색 완료
700페이지 탐색 완료
701페이지 탐색 완료
702페이지 탐색 완료
703페이지 탐색 완료
704페이지 탐색 완료
705페이지 탐색 완료
706페이지 탐색 완료
707페이지 탐색 완료
708페이지 탐색 완료
709페이지 탐색 완료
710페이지 탐색 완료
711페이지 탐색 완료

InvalidSessionIdException: Message: invalid session id
